# Term Project - Milestone #2

Simon Aytes

MAT 328 - Techniques in Data Science

Spring 2022

Professor Owen

----

## Import libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
%matplotlib inline

pd.set_option('display.max_columns', None)

# 1. Load processed data
----

In [14]:
# Get path to the repository's data folder
data_path = "/".join(os.getcwd().split("/")[0:-1]) + "/data/"

In [15]:
chd = pd.read_csv(open(data_path + "processed/data_cleaned.csv"))
chd = chd.drop(columns=["Unnamed: 0"])

In [16]:
chd.head()

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,36,4,1,0,0.0,0,0,1,0,212.0,168.0,98.0,29.77,72.0,75.0,0.0
1,2,46,1,0,1,10.0,0,0,0,0,250.0,116.0,71.0,20.35,88.0,94.0,0.0
2,3,50,1,1,1,20.0,0,0,1,0,233.0,158.0,88.0,28.26,68.0,94.0,1.0
3,4,64,1,0,1,30.0,0,0,0,0,241.0,136.5,85.0,26.42,70.0,77.0,0.0
4,5,61,3,0,0,0.0,0,0,1,0,272.0,182.0,121.0,32.80,85.0,65.0,1.0


# 2. Determining model inputs

----

## Visualize the correlation scores of the dataset features

In [17]:
chd.corr().style.background_gradient(cmap='coolwarm')

,id,age,education,sex,is_smoking,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
id,1.000000,0.024679,-0.029438,-0.011536,-0.015198,-0.031746,0.014426,-0.034277,0.022882,0.004331,-0.015428,0.030634,0.004233,0.033410,0.027590,0.011366,0.010901
age,0.024679,1.000000,-0.161238,-0.034307,-0.204398,-0.184110,0.138570,0.049756,0.313543,0.111085,0.281156,0.405238,0.231322,0.141001,0.013947,0.108882,0.234423
education,-0.029438,-0.161238,1.000000,0.033854,0.030861,0.015506,-0.023534,-0.027410,-0.082675,-0.054464,-0.011575,-0.130417,-0.056034,-0.131304,-0.061501,-0.034099,-0.062442
sex,-0.011536,-0.034307,0.033854,1.000000,0.223871,0.347277,-0.041431,-0.009121,-0.001199,0.007311,-0.074135,-0.046803,0.052869,0.073776,-0.120345,-0.011242,0.081880
is_smoking,-0.015198,-0.204398,0.030861,0.223871,1.000000,0.775714,-0.042033,-0.051558,-0.123369,-0.050831,-0.047703,-0.148528,-0.131062,-0.165179,0.053532,-0.058830,0.033342
cigsPerDay,-0.031746,-0.184110,0.015506,0.347277,0.775714,1.000000,-0.036866,-0.047461,-0.080570,-0.048959,-0.026606,-0.105104,-0.067483,-0.095310,0.055770,-0.064029,0.059797
BPMeds,0.014426,0.138570,-0.023534,-0.041431,-0.042033,-0.036866,1.000000,0.114107,0.260039,0.069443,0.098210,0.280788,0.206209,0.088962,0.014681,0.069125,0.092868
prevalentStroke,-0.034277,0.049756,-0.027410,-0.009121,-0.051558,-0.047461,0.114107,1.000000,0.059703,0.013866,0.002418,0.061712,0.059594,0.029168,-0.018617,0.021427,0.052014
prevalentHyp,0.022882,0.313543,-0.082675,-0.001199,-0.123369,-0.080570,0.260039,0.059703,1.000000,0.082510,0.162100,0.699240,0.612037,0.304790,0.154375,0.086143,0.169136
diabetes,0.004331,0.111085,-0.054464,0.007311,-0.050831,-0.048959,0.069443,0.013866,0.082510,1.000000,0.069232,0.108990,0.058115,0.090249,0.046822,0.617189,0.100006


## View ranked correlations for the `TenYearCHD` response variable

In [25]:
chd.corr().sort_values(by="TenYearCHD", ascending=False)[["TenYearCHD"]].style.background_gradient(cmap='coolwarm')

,TenYearCHD
TenYearCHD,1.000000
age,0.234423
sysBP,0.217419
prevalentHyp,0.169136
diaBP,0.139005
glucose,0.133286
totChol,0.106473
diabetes,0.100006
BPMeds,0.092868
sex,0.081880
